# Imports

In [7]:
#Enable matplotlib to display in jupyter notebook & import it
%matplotlib inline

import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import re
from geopy.geocoders import Nominatim #used in filling missing zipcodes


# Read and Clean Listings.csv

In [8]:
#listings.csv READING

LISTINGS = 'data/listings.csv'

#Choose which columns from the csv to read in.
listings_cols = ['id',
                'host_id',
                'neighbourhood_cleansed',
                'zipcode',
                'latitude',
                'longitude',
                'property_type',
                'room_type',
                'accommodates',
                'bathrooms',
                'amenities',
                'price',
                'cleaning_fee',
                'number_of_reviews',
                'first_review',
                'review_scores_rating',
                'review_scores_accuracy',
                'review_scores_cleanliness',
                'review_scores_checkin',
                'review_scores_communication',
                'review_scores_location',
                'review_scores_value',
                'calculated_host_listings_count',
                'reviews_per_month',
                'bedrooms',
                'beds',
                'cancellation_policy',
                'instant_bookable',
                'minimum_nights'] 

#Read in data from the csv
listings = pd.read_csv(LISTINGS, usecols=listings_cols)

#Rename any Columns as needed
rename_dict = {'id':'listing_id',
              'price':'listed_price'}

listings.rename(columns = rename_dict, inplace=True)

#use listing_id as index
listings.set_index('listing_id', inplace=True)



#############################
#         Cleaning          #
#############################

# 'zipcode' ##########
#Paste this in to zipcode section of cleaning
def latLonToZip(lat, lon):
    '''Take in a latitude and longitude and return the zipcode for that location'''
    geolocator = Nominatim()
    try:
        location = geolocator.reverse(str(lat)+','+str(lon))
        z = re.compile('(\s)([0-9]{5})(,\sUnited)')
        return z.findall(location[0])[0][1]
    except:
        print(str(lat)+','+str(lon),'-----',location)
        return np.nan
    
    print(str(lat)+','+str(lon),'-----',location)
    return np.nan
#Find all missing zippcodes : missing_zipcodes
missing_zipcodes = listings[listings.zipcode.isnull()].copy()

#update rows that are missing zipcodes using latLonToZip to fill missin
listings.zipcode.update(missing_zipcodes.apply(lambda x: latLonToZip(x['latitude'], x['longitude']), axis=1))

#Remove 'zip+4' part of any zipcode 
listings.zipcode = listings.zipcode.apply(lambda x: x[:5])

# 'price' --> 'listed_price' ##########
listings.listed_price = listings.listed_price.replace('[^0-9.]+','',regex=True).astype(float)

# 'cleaning_fee' ##########
listings.cleaning_fee = listings.cleaning_fee.replace('[^0-9.]+','',regex=True).astype(float)

# 'first_review' ##########
listings.first_review = pd.to_datetime(listings.first_review)

# 'amenities' ##########
listings.amenities = listings.amenities.replace('[^\w,\s/]+','',regex=True).apply(lambda x: x.split(','))

# 'instant_bookable' ##########
listings.instant_bookable.replace({'f':False,'t':True}, inplace=True)

#Save to Pickle because it preserves the index and types###################
listings.to_pickle('data/listings_cleaned.pkl')

# Read and Clean Calendar.csv

In [9]:
#Calendar.csv READING

CALENDAR = 'data/calendar.csv'

#Read in all columns from calendar.csv : listing_id, date, available, price
calendar = pd.read_csv(CALENDAR)


#############################
#         Cleaning          #
#############################

# 'date' ##########
calendar.date = pd.to_datetime(calendar.date)

# 'available' ##########
calendar.available.replace({'f':False,'t':True}, inplace=True)

# 'price' ##########
calendar.price = calendar.price.replace('[^0-9.]+','',regex=True).astype(float)

#############################
#         Augmenting        #
#############################

#create column to represent the day of the week for each date
calendar['day_of_week'] = calendar.date.dt.dayofweek

#Fill in missing price values for each listing using mean value for day of week from that listing
calendar.price.fillna(calendar.groupby(['listing_id','day_of_week'])['price'].transform("mean"), inplace=True)

#create column for revenue generate by property (all prices for occupied days are modeled from mean)
calendar['day_revenue'] = np.where(calendar.available, 0.0, calendar.price)

#Save to Pickle because it preserves the index and types
calendar.to_pickle('data/calendar_cleaned.pkl')

# Using Calendar to Augment Listings DataFrame

In [13]:
listings = pd.read_pickle('data/listings_cleaned.pkl')
calendar = pd.read_pickle('data/calendar_cleaned.pkl')

#Create a list of calendars seperated into 4 quarters
quarter_dates = ['2016-09-06','2016-12-06','2017-03-06','2017-06-06','2017-09-06']
q_cal = [calendar[calendar.date.isin(pd.date_range(quarter_dates[n], quarter_dates[n+1]))] for n in range(4)]

#Revnue Per Quarter
for n,q in enumerate(q_cal):
    listings = listings.join(q_cal[n].groupby('listing_id').day_revenue.sum()).rename(columns={'day_revenue':'q'+str(n+1)+'_revenue'})

#Occupancy Per Quarter
for n,q in enumerate(q_cal):
    q_len = len(pd.date_range(quarter_dates[n], quarter_dates[n+1]))
    listings = listings.join((q_len - q_cal[n].groupby('listing_id').available.sum())/q_len).rename(columns={'available':'q'+str(n+1)+'_occupancy_rate'})

listings.to_pickle('data/listing_cleaned.pkl')

# Creating Dummy Variables

In [23]:
amenities = list(set([item for item_list in listings.amenities for item in item_list]) \
         - set(['translation missing enhosting_amenity_49','translation missing enhosting_amenity_50','']))

amn_frame = pd.DataFrame(index = listings.index)

#create the dummy for each amenity and rename the column as you go
for amn in amenities:
    amn_frame = amn_frame.join(listings.amenities.apply(lambda amns: amn in amns)).rename(columns={'amenities':amn})

#listings['analysis_table'] = listings.index
#listings['analysis_table'] = pd.DataFrame(listings.analysis_table.map(lambda x: amn_frame.loc[x]))

In [12]:
analysis_table = pd.merge(listings, amn_frame, left_index=True, right_index=True)

In [24]:
analysis_table = pd.merge(analysis_table, pd.get_dummies(analysis_table.room_type), left_index = True, right_index = True)
analysis_table = pd.merge(analysis_table, pd.get_dummies(analysis_table.property_type), left_index = True, right_index = True)

In [22]:
listings.to_pickle('data/listing_cleaned.pkl')
amenities = list(set([item for item_list in listings.amenities for item in item_list]) \
         - set(['translation missing enhosting_amenity_49','translation missing enhosting_amenity_50','']))
amenities

['Buzzer/Wireless Intercom',
 'Elevator in Building',
 '24Hour Checkin',
 'Safety Card',
 'Hangers',
 'First Aid Kit',
 'Heating',
 'Indoor Fireplace',
 'Pets live on this property',
 'Suitable for Events',
 'Pets Allowed',
 'Smoke Detector',
 'Laptop Friendly Workspace',
 'Washer / Dryer',
 'Doorman',
 'Breakfast',
 'Hot Tub',
 'Kitchen',
 'Fire Extinguisher',
 'Other pets',
 'Gym',
 'Pool',
 'Dryer',
 'Cats',
 'Washer',
 'Internet',
 'Cable TV',
 'Essentials',
 'Wireless Internet',
 'Air Conditioning',
 'Smoking Allowed',
 'Family/Kid Friendly',
 'Carbon Monoxide Detector',
 'Iron',
 'TV',
 'Free Parking on Street',
 'Lock on Bedroom Door',
 'Hair Dryer',
 'Dogs',
 'Shampoo',
 'Wheelchair Accessible',
 'Paid Parking Off Premises',
 'Free Parking on Premises']

In [25]:
cols_to_remove = ['host_id',
                  'zipcode',
                  'neighbourhood_cleansed',
                  'latitude',
                  'longitude', 'amenities', 'first_review', 'room_type','property_type', 'listed_price', 'Carbon Monoxide Detector', 'Entire home/apt','Apartment']
del analysis_data_x['q1_revenue']
del analysis_data_x['q2_revenue']
del analysis_data_x['q3_revenue']
del analysis_data_x['q4_revenue']

del analysis_data_x['review_scores_rating']
del analysis_data_x['review_scores_rating_zscore']

13

In [26]:
set(listings.columns)

{'accommodates',
 'amenities',
 'bathrooms',
 'bedrooms',
 'beds',
 'calculated_host_listings_count',
 'cancellation_policy',
 'cleaning_fee',
 'first_review',
 'host_id',
 'instant_bookable',
 'latitude',
 'listed_price',
 'longitude',
 'minimum_nights',
 'neighbourhood_cleansed',
 'number_of_reviews',
 'property_type',
 'q1_occupancy_rate',
 'q1_revenue',
 'q2_occupancy_rate',
 'q2_revenue',
 'q3_occupancy_rate',
 'q3_revenue',
 'q4_occupancy_rate',
 'q4_revenue',
 'review_scores_accuracy',
 'review_scores_checkin',
 'review_scores_cleanliness',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_rating',
 'review_scores_value',
 'reviews_per_month',
 'room_type',
 'zipcode'}